In [8]:
!pip install kaggle

# Dependencies

In [64]:
import os
import json

from zipfile import ZipFile
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

import warnings
warnings.filterwarnings("ignore")

In [66]:
kaggle_dictionary = json.load(open("data/kaggle.json"))

In [68]:
kaggle_dictionary.keys()

dict_keys(['username', 'key'])

In [70]:
os.environ["KAGGLE_USERNAME"] = kaggle_dictionary["username"]
os.environ["KAGGLE_KEY"] = kaggle_dictionary["key"]

In [72]:
!kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

Dataset URL: https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews
License(s): other




  0%|          | 0.00/25.7M [00:00<?, ?B/s]
100%|##########| 25.7M/25.7M [00:00<00:00, 1.02GB/s]


In [73]:
# unzipping the files
with ZipFile("imdb-dataset-of-50k-movie-reviews.zip", "r") as zip_ref:
  zip_ref.extractall()


# Loading the dataset

In [75]:
data = pd.read_csv("data/IMDB Dataset.csv")

In [76]:
data

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative


In [77]:
data.shape

(50000, 2)

In [78]:
data["sentiment"].value_counts()

sentiment
positive    25000
negative    25000
Name: count, dtype: int64

In [79]:
data.replace({"sentiment": {"positive": 1, "negative": 0}}, inplace=True)

In [87]:
data["sentiment"].value_counts()

sentiment
1    25000
0    25000
Name: count, dtype: int64

## Splitting the dataset

In [92]:
# split data into training data and test data
train_data, test_data = train_test_split(data, test_size=0.2, random_state=2)

In [94]:
print(train_data.shape)
print(test_data.shape)

(40000, 2)
(10000, 2)


In [96]:
# Tokenize text data
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(train_data["review"])
X_train = pad_sequences(tokenizer.texts_to_sequences(train_data["review"]), maxlen=200)
X_test = pad_sequences(tokenizer.texts_to_sequences(test_data["review"]), maxlen=200)

In [104]:
print(X_train)

[[1935    1 1200 ...  205  351 3856]
 [   3 1651  595 ...   89  103    9]
 [   0    0    0 ...    2  710   62]
 ...
 [   0    0    0 ... 1641    2  603]
 [   0    0    0 ...  245  103  125]
 [   0    0    0 ...   70   73 2062]]


In [106]:
print(X_test)

[[   0    0    0 ...  995  719  155]
 [  12  162   59 ...  380    7    7]
 [   0    0    0 ...   50 1088   96]
 ...
 [   0    0    0 ...  125  200 3241]
 [   0    0    0 ... 1066    1 2305]
 [   0    0    0 ...    1  332   27]]


In [108]:
Y_train = train_data["sentiment"]
Y_test = test_data["sentiment"]

## Long short term Memory

In [113]:
# build the model
model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=128, input_length=200))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation="sigmoid"))

In [116]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [118]:
# compile the model
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

## Training the model

In [121]:
model.fit(X_train, Y_train, epochs=5, batch_size=64, validation_split=0.2)

Epoch 1/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 308s 604ms/step - accuracy: 0.7238 - loss: 0.5294 - val_accuracy: 0.8479 - val_loss: 0.3657
Epoch 2/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 306s 612ms/step - accuracy: 0.8303 - loss: 0.3889 - val_accuracy: 0.8612 - val_loss: 0.3389
Epoch 3/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 281s 562ms/step - accuracy: 0.8528 - loss: 0.3512 - val_accuracy: 0.8290 - val_loss: 0.4057
Epoch 4/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 284s 567ms/step - accuracy: 0.8584 - loss: 0.3384 - val_accuracy: 0.8670 - val_loss: 0.3247
Epoch 5/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 297s 594ms/step - accuracy: 0.8927 - loss: 0.2711 - val_accuracy: 0.8698 - val_loss: 0.3166


## Model Evaluation

In [124]:
loss, accuracy = model.evaluate(X_test, Y_test)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")

313/313 ━━━━━━━━━━━━━━━━━━━━ 15s 44ms/step - accuracy: 0.8747 - loss: 0.3062
Test Loss: 0.3033149242401123
Test Accuracy: 0.8762000203132629
